In [1]:
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
from urllib.request import urlopen
import torch.nn as nn
from huggingface_hub import hf_hub_download

import matplotlib.pyplot as plt

# Loading some sources of the projection adapter and image encoder
#hf_hub_download(repo_id="AIRI-Institute/OmniFusion", filename="models.py", local_dir='./')
#from models import CLIPVisionTower

DEVICE = "cuda:1"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_id = "unsloth/llama-2-7b"  # or "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map={"": DEVICE}
)
model.eval()

unk_id = tokenizer.encode("<unk>", add_special_tokens=False)[0]
tokenizer.pad_token_id = 2
tokenizer.eos_token_id = 0

model.resize_token_embeddings(len(tokenizer))
N_EMBEDDINGS = model.model.embed_tokens.weight.shape[0]
print("Number of embeddings in tokenizer:", N_EMBEDDINGS)


[2025-06-12 00:53:56,089] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/.mlspace/envs/vika_kurkin_clone/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jovyan/.mlspace/envs/vika_kurkin_clone/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Number of embeddings in tokenizer: 32000


In [2]:
from datasets import load_dataset

ds = load_dataset("Sayankotor/small_wikipaper")

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/53 [00:00<?, ?it/s]

In [3]:
from torch.utils.data import Dataset
import numpy as np
from ast import literal_eval

class PretrainDataset(Dataset):
    def __init__(self, ds):
        # Ensure 'entities' is parsed if stored as string
        self.ds = [
            item for item in ds['train']
            if len(literal_eval(item['entities'])) > 1
        ]

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        try:
            text = item['text'][:2048]
        except Exception:
            print("Bad example (no text):", item)
            text = "Bad example"
        
        ents = literal_eval(item['entities'])
        embs = np.array(literal_eval(item['entity_embs']))[:200]

        return text, ents, embs

In [4]:
import random
from torch.utils.data import Subset, DataLoader
g = torch.Generator()
g.manual_seed(42)

subset_indices = list(range(16000))
random.shuffle(subset_indices)
dataset = PretrainDataset(ds)
# Wrap the dataset with Subset
subset = Subset(dataset, subset_indices)

# Create the dataloader
dataloader = DataLoader(subset, batch_size=1, shuffle=True)

In [5]:
dataset[5]

('Actresses (Catalan: Actrius) is a 1997 Catalan language Spanish drama film produced and directed by Ventura Pons and based on the award-winning stage play E.R. by Josep Maria Benet i Jornet. The film has no male actors, with all roles played by females. The film was produced in 1996. == Synopsis == In order to prepare herself to play a role commemorating the life of legendary actress Empar Ribera, young actress (Mercè Pons) interviews three established actresses who had been the Ribera\'s pupils: the international diva Glòria Marc (Núria Espert), the television star Assumpta Roca (Rosa Maria Sardà), and dubbing director Maria Caminal (Anna Lizaran). == Cast == * Núria Espert as Glòria Marc * Rosa Maria Sardà as Assumpta Roca * Anna Lizaran as Maria Caminal * Mercè Pons as Estudiant == Recognition == === Screenings === Actrius screened in 2001 at the Grauman\'s Egyptian Theatre in an American Cinematheque retrospective of the works of its director. The film had first screened at the s

### Train adapter

In [ ]:
import torch
from torch import nn
from torch.optim import AdamW
from transformers.optimization import (Adafactor, AdafactorSchedule,
                                       get_cosine_schedule_with_warmup)

import gc

import tqdm

kg_emb_dim = 200
llama_emb_dim = 4096


kg_start_emb = torch.normal(
    torch.zeros(llama_emb_dim), 
    torch.ones(llama_emb_dim) / llama_emb_dim**0.5
).to(device=DEVICE, dtype=torch.bfloat16)

kg_end_emb = torch.normal(
    torch.zeros(llama_emb_dim), 
    torch.ones(llama_emb_dim) / llama_emb_dim**0.5
).to(device=DEVICE, dtype=torch.bfloat16)

projection = nn.Linear(kg_emb_dim, llama_emb_dim).to(device=DEVICE, dtype=torch.bfloat16)

kg_start_emb.requires_grad_()
kg_end_emb.requires_grad_()
model.requires_grad_(False)

    
lr = 5e-3
weight_decay = 1e-3
trainable_parameters = [kg_start_emb] + [kg_end_emb] + list(projection.parameters())

opt = AdamW(trainable_parameters, lr=lr, weight_decay=weight_decay)
loss_fct = nn.CrossEntropyLoss(reduction="none", ignore_index=unk_id)

grad_accum = 256

loss_best = 1000.0

losses = []
losses_batch = []
iters = 0
n_iters = len(dataloader)
scheduler = get_cosine_schedule_with_warmup(opt, num_warmup_steps=n_iters // grad_accum * 0.01, num_training_steps=n_iters // grad_accum)

for epoch in range(1):
    i = 0 
    for step in tqdm.notebook.tqdm(range(n_iters)):
        batch = next(iter(dataloader))
        text, ents, embs = batch
        
        model.eval()
        model.requires_grad = False
        opt.zero_grad()
    
        prompt = f"Continue this text:\n\n{text[0]}"


        with torch.no_grad():
            text_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")[..., :2048].to(device=DEVICE)

            text_ids = tokenizer.encode(prompt, add_special_tokens=False)[:2048]
            
            # Step 2: Add special tokens manually
            text_ids += [tokenizer.eos_token_id]
            
            # Step 3: Convert to tensor
            text_ids = torch.tensor([text_ids], device=DEVICE)
            
            half = text_ids.shape[1] // 2  # use shape[1] for token length
            input_embeddings = model.model.embed_tokens(text_ids[:, :half])
            output_embeddings = model.model.embed_tokens(text_ids[:, half:])
            
        try:
            m = batch[2].mean(2, keepdim=True)
            s = batch[2].std(2, unbiased=False, keepdim=True)
            batch[2] = (batch[2] - m) / (s + 1e-6)
        except:
            print ("except", batch[2].shape)
        try:
            projected_kg_embeddings = projection(batch[2].to(
                        device=DEVICE, dtype=model.dtype
                    ))
        except Exception as e:
            print("❌ Error projecting KG embeddings")
            print("embs.shape:", batch[2].shape)
            print("Exception:", str(e))
            continue
        
        embeddings1 = torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            )
        
        #mask = torch.full(embeddings1.shape, False)
        
        with torch.autocast(device_type="cuda", dtype=model.dtype):
            logits = model(inputs_embeds=torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            ), output_hidden_states=True).get("logits")
            # loss only for answer part & backward

            logits = logits[..., -output_embeddings.shape[1]:-1, :].contiguous()
            labels = text_ids[:, half+1:].contiguous()

            if torch.isnan(logits).any():
                print("⚠️ NaN in logits")
            

            loss = loss_fct(logits.permute(0, 2, 1), labels).mean()

        if model.dtype == torch.float16:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        losses_batch.append(loss.item())
        
        if (step% 1000 == 0):         
            out = model.generate(inputs_embeds=embeddings1, max_new_tokens = half)
            generated_texts2 = tokenizer.batch_decode(out)[0]
            print ("\n first part \n")
            print(tokenizer.decode(text_ids[0, :half].tolist(), skip_special_tokens=True))
            print("\n last part \n")
            print(tokenizer.decode(text_ids[0, half:].tolist(), skip_special_tokens=True))
            print ("\n continue", generated_texts2)
            print ("\n")
            
            print ("loss", np.mean(losses_batch))
            print ('lr', scheduler.get_lr()[0], step, flush = True)
            plt.title("train loss\n" + f"\n\nEpoch [{epoch}], iter [{iters}/{n_iters}]")
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            plt.semilogy(losses)
            plt.grid()
            plt.savefig(f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/loss1_llama2.png")
            plt.close("all")


        if iters % grad_accum == 0 and iters > 0:
            if model.dtype == torch.float16:
                scaler.step(opt)
                scaler.update()
            else:
                opt.step()
            opt.zero_grad()
            scheduler.step()
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            losses_batch = []

            if accum_loss < loss_best:
                loss_best = accum_loss
                torch.save(projection, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/projection_llama2")
                torch.save(kg_start_emb, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/SOI_llama2.pt")
                torch.save(kg_end_emb, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/EOI_llama2.pt")
            
            
            #gc.collect()
        
        iters += 1

        # model inference to get
        
        

## QA sft

In [40]:
import pandas as pd

pd_df = pd.read_csv('/home/jovyan/shares/SR004.nfs2/chekalina/check_halu/83000.csv')

In [41]:
from datasets import load_dataset

from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(pd_df)

tds['question'][99:100]

['Wapack Wilderness and Lake Monomonac share what state?']

In [18]:
def normalize_answer(ans):
    try:
        # Remove punctuation, convert to lowercase
        cleaned = ans.strip().lower().replace("-", " ").replace(",", "")
        # Try to parse as number
        num = w2n.word_to_num(cleaned)
        return str(num)
    except:
        return ans  # leave unchanged if not a number

# Apply to dataset
for ind, elem in enumerate(tds["question"]):
    if "how many" in elem.lower() or "how much" in elem.lower():
        old_ans = tds["answer"][ind]
        new_ans = normalize_answer(old_ans)
        print(f"Q: {elem}\nOriginal: {old_ans} → Normalized: {new_ans}\n")

72998

In [42]:
from torch.utils.data import DataLoader, Dataset
from ast import literal_eval
import numpy as np
import tqdm

import torch
from torch import nn
from torch.optim import AdamW
from transformers.optimization import (Adafactor, AdafactorSchedule,
                                       get_cosine_schedule_with_warmup)


class PretrainDataset(Dataset):
    def __init__(self, ds):
        self.ds = [
            item for item in ds if len(literal_eval(item['ents'])) > 1
        ]


    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        ents = literal_eval(self.ds[idx]['ents'])
        try:
            lst = literal_eval(self.ds[idx]['embs'])
            lst = [elem for elem in lst if elem != -111]
            embs = np.array(lst)
            #embs = np.reshape(embs, (-1, 200)) 
        except:
            print (self.ds[idx]['embs'])
        
        return self.ds[idx]['question'], self.ds[idx]['answer'], ents, embs

In [43]:
import random
from torch.utils.data import Subset, DataLoader
subset_indices = list(range(8000))
random.shuffle(subset_indices)
dataset = PretrainDataset(tds)
# Wrap the dataset with Subset
subset = Subset(dataset, subset_indices)

# Create the dataloader
dataloader = DataLoader(subset, batch_size=1, shuffle=True)

In [44]:
dataset[4][0]

'Ruth, Caldwell County, Kentucky is located near a national forest established in which year ?'

In [45]:
text = tds[2]['answer']
input_ids = tokenizer(text, add_special_tokens=False)['input_ids']
input_ids = input_ids + [tokenizer.eos_token_id]
input_tensor = torch.tensor([input_ids]).to(DEVICE)

In [46]:
input_tensor

tensor([[3082,    0]], device='cuda:1')

In [ ]:
#projection = torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/projection_llama2", map_location=DEVICE)
#kg_start_emb = torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/SOI_llama2.pt", map_location=DEVICE)
#kg_end_emb = torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/EOI_llama2.pt", map_location=DEVICE)

kg_emb_dim = 200
llama_emb_dim = 4096

kg_start_emb = torch.normal(
    torch.zeros(llama_emb_dim), 
    torch.ones(llama_emb_dim) / llama_emb_dim**0.5
).to(device=DEVICE, dtype=torch.bfloat16)

kg_end_emb = torch.normal(
    torch.zeros(llama_emb_dim), 
    torch.ones(llama_emb_dim) / llama_emb_dim**0.5
).to(device=DEVICE, dtype=torch.bfloat16)

projection = nn.Linear(kg_emb_dim, llama_emb_dim).to(device=DEVICE, dtype=torch.bfloat16)

kg_start_emb.requires_grad_()
kg_end_emb.requires_grad_()
projection.requires_grad_()
model.requires_grad_(False)

lr = 5e-3
weight_decay = 1e-3
trainable_parameters = [kg_start_emb] + [kg_end_emb] + list(projection.parameters())

opt = AdamW(trainable_parameters, lr=lr, weight_decay=weight_decay)
loss_fct = nn.CrossEntropyLoss(reduction="none", ignore_index=unk_id)

grad_accum = 256

loss_best = 1000.0

losses = []
losses_batch = []
iters = 0
n_iters = len(dataloader)
scheduler = get_cosine_schedule_with_warmup(opt, num_warmup_steps=n_iters // grad_accum * 0.01, num_training_steps=n_iters // grad_accum)

for epoch in range(1):
    i = 0 
    for step in tqdm.notebook.tqdm(range(n_iters)):
       
        
        batch = next(iter(dataloader))
        question, answer, ents, embs = batch
        ents = [ent[0] for ent in ents]
        
        model.eval()
        model.requires_grad = False
        #opt.zero_grad()
        with torch.no_grad():
            #print (question, answer)
            prompt = f"You are a knowledgeable assistant. Answer the question with a short, simple response. Avoid explanations.\n\n{question[0]}\n\nAnswer:"

            input_ids = tokenizer.encode(prompt, add_special_tokens=False)
            output_ids = tokenizer.encode(answer[0], add_special_tokens=False)
            
            # Step 2: Add special tokens manually
            output_ids += [tokenizer.eos_token_id]
            
            # Step 3: Convert to tensor
            text_ids_in = torch.tensor([input_ids], device=DEVICE)
            text_ids_out = torch.tensor([output_ids], device=DEVICE)

            
            input_embeddings = model.model.embed_tokens(text_ids_in)
            output_embeddings = model.model.embed_tokens(text_ids_out)
            
        if (len(text_ids_out[0]) <= 1):
            continue    
        try:
            # Normalize input embeddings (layer-wise across hidden dim)
            embs = embs.to(device=DEVICE, dtype=model.dtype)
            m = embs.mean(2, keepdim=True)
            s = embs.std(2, unbiased=False, keepdim=True)
            embs = (embs - m) / (s + 1e-6)
        
            # Map to LLM space
            projected_kg_embeddings = projection(embs)
        
            # Optionally re-normalize output to match LLaMA token embedding scale
            # This can help if your LLM embeddings have a stable target variance
            #target_std = 1.0
            #target_mean = 0.0
            #proj_mean = projected_kg_embeddings.mean(dim=-1, keepdim=True)
            #proj_std = projected_kg_embeddings.std(dim=-1, keepdim=True, unbiased=False)
            #projected_kg_embeddings = ((projected_kg_embeddings - proj_mean) / (proj_std + 1e-6)) * target_std + target_mean
        
        except Exception as e:
            print("❌ Projection failed:", e)
            continue
        
        
        #mask = torch.full(embeddings1.shape, False)
        
        with torch.autocast(device_type="cuda", dtype=model.dtype):
            logits = model(inputs_embeds=torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            ), output_hidden_states=True).get("logits")
            
            num_output_tokens = output_embeddings.shape[1]
            #print ("logits.shape", logits.shape)
            logits = logits[..., -1-output_embeddings.shape[1]:-1, :].contiguous()
            labels = text_ids_out.contiguous()
            #print ("logits.shape", logits.shape)
            #print ("labels.shape", labels.shape)
            
            loss = loss_fct(logits.permute(0, 2, 1), labels).mean()
            losses_batch.append(loss.item())
            
            if (labels >= logits.shape[-1]).any():
                print("❌ Labels contain values outside valid range!")
            if torch.isnan(logits).any():
                print("⚠️ NaN in logits")
            if torch.isnan(text_ids_out).any():
                print("⚠️ NaN in labels")

            
        if (step% 1000 == 0): 
            embeddings1 = torch.cat(
                [
                    input_embeddings,
                    kg_start_emb[None, None, ...],
                    projected_kg_embeddings,
                    kg_end_emb[None, None, ...],
                    output_embeddings
                ],
                dim=1,
            )
            print ("ents", ents)
            out = model.generate(inputs_embeds=embeddings1, max_new_tokens = embeddings1.shape[1])
            generated_texts = tokenizer.batch_decode(out)[0]
            print (tokenizer.batch_decode(text_ids_in)[0])
            print ("\n last part \n")
            print (tokenizer.batch_decode(text_ids_out)[0])
            print ("\n continue", generated_texts)
            print ("\n")
            
            print ("loss", np.mean(losses_batch))
            plt.title("train loss\n" + f"\n\nEpoch [{epoch}], iter [{iters}/{n_iters}]")
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            plt.semilogy(losses)
            plt.grid()
            plt.savefig(f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/loss3.png")
            plt.close("all")

        if model.dtype == torch.float16:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        


        if iters % grad_accum == 0 and iters > 0:
            if model.dtype == torch.float16:
                scaler.step(opt)
                scaler.update()
            else:
                opt.step()
            opt.zero_grad()
            scheduler.step()
            accum_loss = np.mean(losses_batch)
            losses.append(accum_loss)
            losses_batch = []

            if accum_loss < loss_best:
                loss_best = accum_loss
                torch.save(projection, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/projection_llama2_qa")
                torch.save(kg_start_emb, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/SOI_llama2_qa.pt")
                torch.save(kg_end_emb, f"/home/jovyan/shares/SR004.nfs2/chekalina/kg_reduces_halus/notebook_new/ckpts/EOI_llama2_qa.pt")
            
            
            #gc.collect()
        
        iters += 1

        
        

  0%|          | 0/8000 [00:00<?, ?it/s]

ents ['Libocedrus', 'Helianthus']
You are a knowledgeable assistant. Answer the question with a short, simple response. Avoid explanations.

Are Libocedrus and Helianthus both types of plants?

Answer:

 last part 

yes<unk>

 continue 

Comment: Are Libocedrus and Helianthus both types of plants?

Answer: Are Libocedrus and Helianthus both types of plants?

Answer: Yes

Comment: Are Lib


loss 2.984375
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of range (expected to be in range of [-2, 1], but got 2)
❌ Projection failed: Dimension out of ran

In [ ]:
tok_ids = tokenizer("The capital of France is", return_tensors="pt").input_ids.to(device)
embs = model.model.embed_tokens(tok_ids)
print(embs.mean().item(), embs.std().item())

In [12]:
import os
import torch
import collections
import re
from typing import Union, List
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm

def normalize_and_tokenize_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip().lower().split()

def calculate_em(pred: Union[list, str, None], answer: Union[list, str, None], mode: str) -> int:
    if mode == 'text' and pred and answer:
        pred = normalize_and_tokenize_text(pred)
        answer = normalize_and_tokenize_text(answer)
        print (pred, answer)
        for i in range(0, len(pred) - len(answer) + 1):
            print (answer, pred[i: i + len(answer)])
            if answer == pred[i: i + len(answer)]:
                return 1
        return 0
    else:
        return int(pred == answer)




In [13]:
calculate_em('27', '2', 'text')

['27'] ['2']
['2'] ['27']


0